# RAG評価用データセットの生成
* Ragasでは、評価用のデータセットを生成できる
* 以下、作成方法
* 書籍のバージョンではデータセットに設定するキー名が旧版のためエラーがでるので注意

In [1]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.10.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: Regression. AzureChatOpenAI json-mode broken. Fixed in 0.1.22.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 24.6 MB/s eta 

In [2]:
# # 環境変数(.env)
# OPENAI_API_KEY=
# LANGCHAIN_API_KEY=
# LANGCHAIN_PROJECT=

In [3]:
import os
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [5]:
### ベクトル検索対象データ用意
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

280


## * Ragasで合成データ作成

In [6]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]


nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

testset.to_pandas()

embedding nodes:   0%|          | 0/902 [00:00<?, ?it/s]

Generating:   0%|          | 0/4 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What features are provided by the instant mess...,[# Telegram\n\n>[Telegram Messenger](https://w...,Telegram provides features such as end-to-end ...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What is the purpose of creating an API token w...,[# PromptLayer\n\n>[PromptLayer](https://docs....,The purpose of creating an API token when work...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,How does MLflow simplify LLM provider interact...,[# MLflow Deployments for LLMs\n\n>[The MLflow...,MLflow simplifies LLM provider interactions by...,reasoning,[{'source': 'docs/docs/integrations/providers/...,True
3,How does LLM observability boost security in L...,[# PromptLayer\n\n>[PromptLayer](https://docs....,The answer to given question is not present in...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


### * LangSmithのDataSet作成
* LangSmithには、評価用の「Dataset」を管理する機能がある。

#### 以下は、LangSmithでデータセットを管理する「Dataset」オブジェクトを作成するコード

In [7]:
from langsmith import Client

dataset_name = "test-dataset-2"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(
    dataset_name=dataset_name,
)

上記実行後、LangSmithの画面で、『Datasets & Experiments』を選ぶと"test-dataset"というデータセットができる。ここに、Ragasで生成した合成テストデータを保存する

### * 合成テストデータの保存
* まずは、生成したデータセットをLangSmithのDatasetに保存する形式に変換

In [8]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            # "question": testset_record.question, # 古いバージョン
            "user_input": testset_record.question,
        }
    )
    outputs.append(
        {
            # "contexts": testset_record.contexts,
            # "ground_truth": testset_record.ground_truth,
            "retrieved_contexts": testset_record.contexts,      # ← contexts → retrieved_contexts
            "response": testset_record.ground_truth,            # ← ground_truth → response
            "reference": testset_record.ground_truth,           # ← 必要に応じて reference も登録
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

### * LangSmithのクライアントを使用して、DatasetのIDを指定しデータを保存
* LangSmithでは、Datasetに保存するデータの１件１件を「Example」と呼ぶ。


In [9]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadatas=metadatas,
    dataset_id=dataset.id,
)

* これにより、web画面で確認したDataset（"test-dataset")を選ぶと、登録されているのがわかる